### Praktikum 2

In [33]:
import tensorflow as tf # digunakan untuk membangun dan melatih model machine learning
import numpy as np # digunakan untuk menangani array multidimensi dan operasi matematis
import os # digunakan untuk berinteraksi dengan sistem operasi
import time # digunakan untuk mengukur waktu

In [34]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt') # mengunduh file dari URL

In [35]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8') # digunakan untuk membaca file teks yang dikodekan dalam UTF-8 dan mengembalikan konten file tersebut sebagai string
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters') # mencetak panjang dari string text

Length of text: 1115394 characters


In [36]:
print(text[:250]) # Menampilkan 250 karakter pertama

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [37]:
vocab = sorted(set(text)) # membuat vocab dengan set dari teks yang diurutkan
print(f'{len(vocab)} unique characters') # menampilkan panjang variabel vocab

65 unique characters


Mengolah Teks

In [38]:
example_texts = ['abcdefg', 'xyz'] # Membuat list example_texts
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8') # memecah teks yang diberikan menjadi urutan poin kode Unicode
chars # menampilkan variabel chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [39]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None) # membuat lapisan StringLookup TensorFlow yang dapat digunakan untuk mengkonversi token teks menjadi ID numerik

In [40]:
ids = ids_from_chars(chars) # mengkonversi urutan poin kode Unicode chars menjadi urutan ID numerik menggunakan lapisan StringLookup
ids # Menampilkan variabel ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [41]:
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None) # membuat lapisan StringLookup TensorFlow dengan tambahan argument invert=True dan tidak menggunakan token mask

In [42]:
chars = chars_from_ids(ids) # mengkonversi urutan poin kode Unicode chars menjadi urutan ID numerik menggunakan lapisan StringLookup
chars # Menampilkan variabel ids

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [43]:
tf.strings.reduce_join(chars, axis=-1).numpy() # menggabungkan nilai pada chars menjadi string

array([b'abcdefg', b'xyz'], dtype=object)

In [44]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1) # mengembalikan string yang digabungkan dari semua token teks dalam urutan ID numerik ids

Prediksi - Membuat Training Set dan Target

In [45]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8')) # mengkonversi semua karakter dalam teks text menjadi ID numerik menggunakan lapisan StringLookup ids_from_chars
all_ids # Menampilkan variabel

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [46]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids) # membuat dataset TensorFlow dari urutan ID numerik all_ids

In [47]:
for ids in ids_dataset.take(10): # Melakukan perulangan pada 10 elemen pertama
    print(chars_from_ids(ids).numpy().decode('utf-8')) #  mencetak token teks yang sesuai dengan ID numerik ids

F
i
r
s
t
 
C
i
t
i


In [48]:
seq_length = 100 # Inisialisasi variabel seq_length

In [49]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True) # membuat batch tensor ID numerik dari dataset ids_dataset, dengan ID urutan selanjutnya sebagai target dan mengabaikan batch terakhir jika panjangnya kurang dari seq_length+1

for seq in sequences.take(1): # Melakukan perulangan nilai pertama
  print(chars_from_ids(seq)) # mencetak token teks yang sesuai dengan urutan ID

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [50]:
for seq in sequences.take(5): # Melakukan perulangan pada 5 elemen pertama
    print(text_from_ids(seq).numpy()) # mencetak teks yang sesuai dengan urutan ID numerik seq

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [51]:
def split_input_target(sequence):
    input_text = sequence[:-1] # mengambil seluruh nilai kecuali nilai terakhir
    target_text = sequence[1:] # mengambil seluruh nilai kecuali nilai pertama
    return input_text, target_text # mengembalikan variabel input_text dan target_text

In [52]:
split_input_target(list("Tensorflow")) # menjalankan function split_input_target dengan argument list 'Tensorflow'

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [53]:
dataset = sequences.map(split_input_target) # memetakan setiap elemen dalam dataset sequences ke urutan ID numerik yang berisi input teks dan target teks

In [54]:
for input_example, target_example in dataset.take(1): # melakukan perulangan pada elemen pertama dataset dan membaginya ke variabel input_example dan target_example
    print("Input :", text_from_ids(input_example).numpy()) # menampilkan nilai input_example
    print("Target:", text_from_ids(target_example).numpy()) # menampilkan nilai target_example

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


Membuat Bath Training

In [55]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000 # Inisialisasi variabel BUFFER_SIZE

dataset = (
    dataset # Membuat dataset
    .shuffle(BUFFER_SIZE) # mengacak urutan elemen dalam dataset dan menyimpan elemen-elemen yang diacak berdasarkan buffer_size
    .batch(BATCH_SIZE, drop_remainder=True) # mengelompokkan elemen-elemen dalam dataset menjadi batch dan  elemen-elemen yang tidak dapat dibagi oleh BATCH_SIZE akan dibuang
    .prefetch(tf.data.experimental.AUTOTUNE)) # TensorFlow secara otomatis menentukan ukuran buffer yang optimal untuk prefetch

dataset # menampilkan variabel dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

Membuat Model

In [56]:
vocab_size = len(ids_from_chars.get_vocabulary()) # menghitung ukuran vocabulary dari lapisan StringLookup ids_from_chars

embedding_dim = 256 # Inisialisasi variabel embedding_dim

rnn_units = 1024 # Inisialisasi variabel rnn_units

In [57]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim) # membuat embedding layer untuk memproses urutan ID numerik yang dihasilkan oleh lapisan StringLookup
    self.gru = tf.keras.layers.GRU(rnn_units, # Jumlah unit sebanyak rnn_units
                                   return_sequences=True, # mengembalikan urutan vektor numerik yang sama dengan panjang urutan input
                                   return_state=True) # membuat lapisan GRU untuk memproses urutan vektor numerik yang dihasilkan oleh lapisan embedding dan mengembalikan keadaan internal dari lapisan GRU
    self.dense = tf.keras.layers.Dense(vocab_size) # membuat layer dense dengan argument vacab_size

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs # menyimpan variabel inputs ke dalam variabel x
    x = self.embedding(x, training=training) # memproses urutan ID numerik x menggunakan lapisan embedding self.embedding dengan set mode training berdasarkan variabel training
    if states is None:
      states = self.gru.get_initial_state(x) # mendapatkan keadaan awal dari lapisan GRU
    x, states = self.gru(x, initial_state=states, training=training) # memproses urutan vektor numerik x menggunakan lapisan GRU self.gru dengan keadaan awal states dan mode berdasarkan variabel training
    x = self.dense(x, training=training) # memproses urutan vektor numerik x menggunakan lapisan dense

    if return_state:
      return x, states # mengembalikan nilai x dan states
    else:
      return x # mengembalikan nilai x

In [58]:
model = MyModel( # Memanggil function MyModel()
    vocab_size=vocab_size, # menambahkan argument berupa variabel vocab_size
    embedding_dim=embedding_dim, # menambahkan argument berupa variabel embedding_dim
    rnn_units=rnn_units) # menambahkan argument berupa variabel rnn_units

Menguji Model

In [59]:
for input_example_batch, target_example_batch in dataset.take(1): # Melakukan perulangan dengan elemen pertama yang dibagi dalam variabel input_example_batch dan target_example_batch
    example_batch_predictions = model(input_example_batch) # menghasilkan prediksi untuk batch input input_example_batch menggunakan model
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)") # menampilkan ukuran example_batch_predictions

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [60]:
model.summary() # mencetak ringkasan dari model machine learning yang telah dibuat

Model: "my_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     multiple                  16896     
                                                                 
 gru_2 (GRU)                 multiple                  3938304   
                                                                 
 dense_2 (Dense)             multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [61]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1) # mengambil sampel dari distribusi kategorikal yang ditentukan oleh vektor probabilitas example_batch_predictions[0] dengan jumlah sampel yang diambil adalah 1
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy() # menghapus dimensi dari tensor dan mengubah tensor sampled_indices menjadi array NumPy

In [62]:
sampled_indices # menampilkan array variabel sampled_indices

array([24, 45,  0, 26, 55, 54, 42, 49,  3, 55, 49, 60,  1, 44, 63, 47, 54,
       22, 25, 43, 10,  2, 64, 23, 13, 52, 42, 14, 41, 64,  2, 46, 55,  4,
       30, 60, 14, 46, 20, 49, 65, 26, 54, 13, 61, 49, 22, 15, 13, 34, 35,
       35, 47, 15, 39, 22, 47, 27, 43, 19, 23,  6, 44, 60, 17, 50, 60, 37,
       43,  8,  2, 30, 14, 27, 55, 39, 10, 62, 39, 12, 61, 23, 53, 60, 53,
        5, 11, 43, 21,  9,  6, 13, 62, 22, 17, 57,  8, 26, 56, 30])

In [63]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy()) # mencetak input pertama dalam batch input_example_batch
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy()) # mencetak prediksi karakter berikutnya untuk input pertama dalam batch input_example_batch

Input:
 b't banish reason\nFor inequality; but let your reason serve\nTo make the truth appear where it seems hi'

Next Char Predictions:
 b"Kf[UNK]Mpocj!pju\nexhoILd3 yJ?mcAby gp$QuAgGjzMo?vjIB?UVVhBZIhNdFJ'euDkuXd- QANpZ3wZ;vJnun&:dH.'?wIDr-MqQ"


Train Model

In [64]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True) # membuat fungsi kerugian Sparse Categorical Crossentropy dengan menerima logits sebagai input

In [66]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions) # menghitung rata-rata kerugian untuk batch input example_batch_predictions
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") # menampilkan ukuran example_batch_predictions
print("Mean loss:        ", example_batch_mean_loss) # menampilkan mean loss

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1892314, shape=(), dtype=float32)


In [67]:
tf.exp(example_batch_mean_loss).numpy() # menghitung eksponen dari rata-rata kerugian untuk batch input

65.97207

In [68]:
model.compile(optimizer='adam', loss=loss) # Compile model denga optimizer ADAM dan argument variabel loss

Konfigurasi Checkpoint

In [69]:
checkpoint_dir = './training_checkpoints' # membuat direktori untuk menyimpan checkpoint model

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}") # membuat prefix nama file untuk checkpoint model

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True) # menyimpan checkpoint model machine learning ke direktori checkpoint_dir dengan prefix nama file ckpt_{epoch} dan yang disimpan hanya berupa bobotnya


Melakukan Proses Training

In [70]:
EPOCHS = 20 # Inisialisasi variabel EPOCHS

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback]) # melatih model  dengan dataset dataset selama sebanyak EPOCHS, dengan menggunakan callback untuk menyimpan checkpoint model

Epoch 1/20
172/172 [==============================] - 15s 51ms/step - loss: 2.7140
Epoch 2/20
172/172 [==============================] - 11s 51ms/step - loss: 1.9813
Epoch 3/20
172/172 [==============================] - 10s 50ms/step - loss: 1.7065
Epoch 4/20
172/172 [==============================] - 10s 51ms/step - loss: 1.5477
Epoch 5/20
172/172 [==============================] - 11s 51ms/step - loss: 1.4484
Epoch 6/20
172/172 [==============================] - 10s 51ms/step - loss: 1.3803
Epoch 7/20
172/172 [==============================] - 10s 51ms/step - loss: 1.3283
Epoch 8/20
172/172 [==============================] - 11s 52ms/step - loss: 1.2827
Epoch 9/20
172/172 [==============================] - 12s 52ms/step - loss: 1.2423
Epoch 10/20
172/172 [==============================] - 11s 53ms/step - loss: 1.2029
Epoch 11/20
172/172 [==============================] - 11s 52ms/step - loss: 1.1623
Epoch 12/20
172/172 [==============================] - 11s 52ms/step - loss: 1.1219
E

Generate Teks

In [71]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [72]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars) # membuat model one-step untuk menghasilkan teks

In [73]:
start = time.time() # mencatat waktu mulai dari suatu proses
states = None # Memberikan nilai awal None pada states
next_char = tf.constant(['ROMEO:']) # membuat tensor konstan yang berisi karakter ROMEO, yang nilainya tidak dapat diubah
result = [next_char] # membuat list yang berisi tensor konstan

for n in range(1000): # Melakukan 1000 perulangan
  next_char, states = one_step_model.generate_one_step(next_char, states=states) # menghasilkan karakter berikutnya dari input teks menggunakan model one-step one_step_model
  result.append(next_char) # Menambahkan nilai next_char ke list result

result = tf.strings.join(result) # menggabungkan semua elemen dari list result menjadi satu string.
end = time.time() # mencatat waktu proses berakhir
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80) # mencetak teks yang dihasilkan oleh model one-step, diikuti oleh garis bawah sepanjang 80 karakter dalam format decode utf-8
print('\nRun time:', end - start) # Menampilkan waktu runtime dari selisih waktu mulai dan selesai

ROMEO:
Then most never o'erwear him to her life?

JULIET:
To let it is for policy.

FLORIZEL:
I needs must go till.

A Fige. But, O, he'l oft,
And hath a harm in't then comes shake it.

LUCENTIO:
And what good morning without bears more tears.

LADY GREY:
Methinks I would I were i' the laughtain wears;
Go, bill, a crow, both honour now in peace:
Beseech you, so your free contents go you in such a righter bed.

First Murderer:
Thou hadst an body should be that so set door.
3 KING HENRY VI

WARDINA:
How! nurse?

Second Senator:
He cannot: we
Will have them yet.
Ah, what a sceptre's heir
Of lambs praised their hell, as brought
To this so-noise-mour's wife in hand: so, take my revenge
Are all frants for joy or to this mass, Good, Sir boistero
As far as many fears time thus hard-teaching nature,
Unlikely longes to encounter? his channing
may catching.

ROMEO:
Thou wilt amend thy brain; And now I care
not right what raised the happy evil intent:
Acquaint the king in Padua for any
often I wit

In [74]:
start = time.time() # mencatat waktu mulai dari suatu proses
states = None # Memberikan nilai awal None pada states
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:']) # membuat tensor konstan yang berisi lima karakter ROMEO, yang nilainya tidak dapat diubah
result = [next_char] # membuat list yang berisi tensor konstan

for n in range(1000): # Melakukan 1000 perulangan
  next_char, states = one_step_model.generate_one_step(next_char, states=states) # menghasilkan karakter berikutnya dari input teks menggunakan model one-step one_step_model
  result.append(next_char) # Menambahkan nilai next_char ke list result

result = tf.strings.join(result) # menggabungkan semua elemen dari list result menjadi satu string.
end = time.time() # mencatat waktu proses berakhir
print(result, '\n\n' + '_'*80) # mencetak teks yang dihasilkan oleh model one-step, diikuti oleh garis bawah sepanjang 80 karakter
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80) # mencetak teks yang dihasilkan oleh model one-step, diikuti oleh garis bawah sepanjang 80 karakter  dalam format decode utf-8
print('\nRun time:', end - start) # Menampilkan waktu runtime dari selisih waktu mulai dan selesai

tf.Tensor(
[b"ROMEO:\nFair lips, a doble confine at my back,\nThat are being begs at my request;\nAnd I, whomas I have senter'd there were\nwindows tatch over Clarence, we will dost thou nother.\n\nGLOUCESTER:\nThou dost take no conceality, and bear himself\nAs honourable lord chamberlalo!\n\nDORSET:\nMy lenJamenta's boty subjects now.\nMy lord lessed was, he bear me: in paunt a troubly\nYou have frown'd upon my heart that late.\n\nMENENIUS:\nI was not my good lord; since that gleat'st not this feat?\nOur hope more, fair Bopierey.' 'Now you, catched, great;\nThe boar will revenge upon you are all born,\nBeside your wisdoms, my father, if thou wast\nAs sorry the earthly gatherous sound;\nAnd, every sleep and executioner, so took of him\nA dismal throat, shall we to A woman's tear?\n\nTuts,\nLet me do soe! thy father is amorout,\nHere will day in his head: the senate power's\nBunkind-like replord; soon am I ne'er advertised\nThat, good feeble and for require;\nAnd at that clife, and that

Ekspor Model Generator

In [75]:
tf.saved_model.save(one_step_model, 'one_step') # menyimpan model one-step one_step_model ke direktori one_step
one_step_reloaded = tf.saved_model.load('one_step') # memuat model one-step yang disimpan di direktori one_step ke dalam variabel one_step_reloaded

In [76]:
states = None # Memberikan nilai awal None pada states
next_char = tf.constant(['ROMEO:']) # membuat tensor konstan yang berisi karakter ROMEO, yang nilainya tidak dapat diubah
result = [next_char] # membuat list yang berisi tensor konstan

for n in range(100): # Melakukan 100 perulangan
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states) # menghasilkan karakter berikutnya dari input teks menggunakan model one-step one_step_model
  result.append(next_char) # Menambahkan nilai next_char ke list result

print(tf.strings.join(result)[0].numpy().decode("utf-8")) # # menggabungkan semua elemen dari list result menjadi satu string dan menampilkan hasilnya dalam format decode utf-8

ROMEO:
The friend aforements the old man of time past wrong'd,
Which restrained gown to be his wife anothe
